In [3]:
%load_ext autoreload
import cobra
import numpy as np
from importlib import reload
import mcs
from scipy import sparse
#import traceback
#import warnings
import sys


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


This is a bilevel-example with 1 Target and 1 Desired region

In [25]:
%autoreload
# load network
network = cobra.io.read_sbml_model("weak_coupling.sbml")
mcs.fba(network,constr=["r1=0"])
# specify modules
# specify modules
modules = [mcs.SD_Module(network,"mcs_bilvl",module_sense="target",constraints=["r_P <= 0.5"],inner_objective="-r_BM")]
modules.append(mcs.SD_Module(network,"mcs_lin",module_sense="desired",constraints="r_BM >= 1"))

# specify MCS setup
ko_cost = {'r2': 1, 'r3': 1, 'r4': 1, 'r5': 1, 'r6': 1, 'r7': 1, 'r8': 1, 'r9': 1}
ki_cost = {'r1': 1}
maxSolutions = 50
maxCost = 50

# construct MCS MILP
mcsEnum = mcs.StrainDesigner(network,modules,ko_cost=ko_cost,ki_cost=ki_cost,max_cost=7,M=None)

# solve MILP
mcsEnum.enumerate_mcs()
# mcsEnum.compute_mcs()
# mcsEnum.compute_smallest_mcs()

Constructing MCS MILP.
Bounding MILP.
Enumerating MCS ...
MCS with cost 4.0: [{'r1': 1.0, 'r4': -1.0, 'r7': -1.0, 'r9': -1.0}]
MCS with cost 4.0: [{'r1': 1.0, 'r5': -1.0, 'r7': -1.0, 'r9': -1.0}]
Finished. 2 solutions found.


([{'r1': 1.0, 'r4': -1.0, 'r7': -1.0, 'r9': -1.0},
  {'r1': 1.0, 'r5': -1.0, 'r7': -1.0, 'r9': -1.0}],
 0)

This is a bilevel-example with only 1 desired region

In [3]:
%autoreload
# load network
network = cobra.io.read_sbml_model("weak_coupling.sbml")
mcs.fba(network,constr=["r1=0"])
# specify modules
modules = mcs.SD_Module(network,"mcs_bilvl",module_sense="desired",constraints=["r_BM >= 0.5","r_P >= 1"],inner_objective="-r_BM")

# specify MCS setup
ko_cost = {'r2': 1, 'r3': 1, 'r4': 1, 'r5': 1, 'r6': 1, 'r7': 1, 'r8': 1, 'r9': 1}
ki_cost = {'r1': 1}
maxSolutions = 50
maxCost = 50

# construct MCS MILP
mcsEnum = mcs.StrainDesigner(network,modules,ko_cost=ko_cost,ki_cost=ki_cost,max_cost=7,M=None)

# solve MILP
mcsEnum.compute_smallest_mcs()

Constructing MCS MILP.
Bounding MILP.
Enumerating smallest MCS ...
{'r1': 1, 'r5': -1, 'r7': -1}
{'r1': 1, 'r4': -1, 'r7': -1}


(<2x13 sparse matrix of type '<class 'numpy.float64'>'
 	with 6 stored elements in Compressed Sparse Row format>,
 0)

This is a regular MCS example with 1 target and 1 desired region

In [33]:
%autoreload
# load network
network = cobra.io.read_sbml_model("SmallExample.sbml")

# specify modules
modules = [mcs.SD_Module(network,"mcs_lin",module_sense="target",constraints=["R04 >= 1","2 R01 >= 0.5"])]
modules.append(mcs.SD_Module(network,"mcs_lin",module_sense="desired",constraints="R03 >= 1"))

# specify MCS setup
ko_cost = {'R05': 2, 'R06': 3, 'R07': 4, 'R08': 4, 'R09': 4, 'R10': 4}
ki_cost = {'R02': 6}
maxSolutions = 8
maxCost = 8

# construct MCS MILP
mcsEnum = mcs.StrainDesigner(network,modules,ko_cost=ko_cost,ki_cost=ki_cost,max_cost=maxCost,M=None)

# solve MILP
# mcsEnum.enumerate_mcs()
mcsEnum.compute_mcs()
# mcsEnum.compute_smallest_mcs()

Constructing MCS MILP.
Bounding MILP.
Finding MCS of arbitrary size ...
Searching in full search space.
Searching in subspace with 2 possible targets.
MCS with cost 4: {'R07': -1}
Searching in full search space.
Searching in subspace with 2 possible targets.
MCS with cost 4: {'R10': -1}
Searching in full search space.
Searching in subspace with 2 possible targets.
MCS with cost 7: {'R06': -1, 'R08': -1}
Searching in full search space.
Finished. 3 solutions found.


([{'R07': -1.0}, {'R10': -1.0}, {'R06': -1.0, 'R08': -1.0}], 0)

This is a bilevel-example with 1 Target and 1 Desired region

In [40]:
%autoreload
# load network
network = cobra.io.read_sbml_model("weak_coupling.sbml")
mcs.fba(network,constr=["r1=0"])
# specify modules
# specify modules
modules = mcs.SD_Module(network,"mcs_bilvl",module_sense="target",constraints=["r_P <= 0.5"],inner_objective="-r_BM")

# specify MCS setup
maxSolutions = 10
maxCost = 2

# construct MCS MILP
mcsEnum = mcs.StrainDesigner(network,modules,max_cost=maxCost,M=None)

# solve MILP
# mcsEnum.compute_mcs(max_solutions=maxSolutions)
mcsEnum.enumerate_mcs(max_solutions=maxSolutions)
# mcsEnum.compute_smallest_mcs(max_solutions=maxSolutions)

Constructing MCS MILP.
Bounding MILP.
Enumerating MCS ...
MCS with cost 1.0: [{'r2': -1.0}]
MCS with cost 1.0: [{'r_S': -1.0}]
MCS with cost 2.0: [{'r_BM': -1.0, 'r_Q': -1.0}]
MCS with cost 2.0: [{'r3': -1.0, 'r_BM': -1.0}]
MCS with cost 2.0: [{'r4': -1.0, 'r_BM': -1.0}]
MCS with cost 3.0: [{'r4': -1.0, 'r7': -1.0, 'r_Q': -1.0}]
MCS with cost 3.0: [{'r5': -1.0, 'r6': -1.0, 'r_Q': -1.0}]
MCS with cost 3.0: [{'r5': -1.0, 'r7': -1.0, 'r9': -1.0}]
MCS with cost 3.0: [{'r4': -1.0, 'r7': -1.0, 'r9': -1.0}]
MCS with cost 3.0: [{'r4': -1.0, 'r6': -1.0, 'r_Q': -1.0}]
Finished. 10 solutions found.


([{'r2': -1.0},
  {'r_S': -1.0},
  {'r_BM': -1.0, 'r_Q': -1.0},
  {'r3': -1.0, 'r_BM': -1.0},
  {'r4': -1.0, 'r_BM': -1.0},
  {'r4': -1.0, 'r7': -1.0, 'r_Q': -1.0},
  {'r5': -1.0, 'r6': -1.0, 'r_Q': -1.0},
  {'r5': -1.0, 'r7': -1.0, 'r9': -1.0},
  {'r4': -1.0, 'r7': -1.0, 'r9': -1.0},
  {'r4': -1.0, 'r6': -1.0, 'r_Q': -1.0}],
 0)

This is a mid-scale-example with only 1 desired region

In [6]:
%autoreload
# load network
network = cobra.io.read_sbml_model("ECC2comp.sbml")
# remove external metabolites and obsolete reactions
external_mets = [i for i,cpts in zip(network.metabolites,network.metabolites.list_attr("compartment")) if cpts == 'External_Species']
network.remove_metabolites(external_mets)
S = cobra.util.create_stoichiometric_matrix(network)
obsolete_reacs = [reac for reac,b_rempty in zip(network.reactions,np.any(S,0)) if not b_rempty]
network.remove_reactions(obsolete_reacs)
# close some uptakes
network.reactions.AcUp.upper_bound = 0
network.reactions.GlycUp.upper_bound = 0

sol = mcs.fba(network,constr=["O2Up=0"])
# specify modules
modules = mcs.SD_Module(network,"mcs_bilvl",module_sense="desired",constraints=["2 Growth >= 0.2","EthEx >= 1"],inner_objective="-Growth")

network.remove_metabolites
network.metabolites.list_attr("compartment")
# specify MCS setup
maxSolutions = 50
maxCost = 50

# construct MCS MILP
mcsEnum = mcs.StrainDesigner(network,modules,max_cost=7,M=None)

# solve MILP
mcsEnum.compute_smallest_mcs(max_solutions=maxSolutions,time_limit=5)

Constructing MCS MILP.
Bounding MILP.
Enumerating smallest MCS ...
{'R_CYTBO3_4pp': -1}
{'R_O2Up': -1}
{'R_h2oEx': -1}
{'R_ATPS4rpp': -1, 'R_SUCCt3pp': -1, 'R_THD2pp': -1}
{'R_G3PD5': -1, 'R_NADH16pp': -1, 'R_NADH17pp': -1}
{'R_ATPS4rpp': -1, 'R_SUCCt2_2pp': -1, 'R_THD2pp': -1}
{'R_ACKr': -1, 'R_ATPS4rpp': -1, 'R_CO2Ex': -1}
{'R_GLCt2pp': -1, 'R_MDH': -1, 'R_PPS': -1}
{'R_ACKr': -1, 'R_AKGDH': -1, 'R_ATPS4rpp': -1}
{'R_AKGDH': -1, 'R_ATPS4rpp': -1, 'R_PTAr': -1}
{'R_ATPS4rpp': -1, 'R_CO2Ex': -1, 'R_PTAr': -1}
{'R_AcEx': -1, 'R_ATPS4rpp': -1, 'R_MDH': -1}
{'R_MDH': -1, 'R_PPS': -1, 'R_TPI': -1}


(<13x82 sparse matrix of type '<class 'numpy.float64'>'
 	with 33 stored elements in Compressed Sparse Row format>,
 3)

In [7]:
%autoreload
# load network
network = cobra.io.read_sbml_model("ECC2comp.sbml")
# remove external metabolites and obsolete reactions
external_mets = [i for i,cpts in zip(network.metabolites,network.metabolites.list_attr("compartment")) if cpts == 'External_Species']
network.remove_metabolites(external_mets)
S = cobra.util.create_stoichiometric_matrix(network)
obsolete_reacs = [reac for reac,b_rempty in zip(network.reactions,np.any(S,0)) if not b_rempty]
network.remove_reactions(obsolete_reacs)
# close some uptakes
network.reactions.AcUp.upper_bound = 0
network.reactions.GlycUp.upper_bound = 0
network.reactions.ATPM.upper_bound = 1000
network.reactions.ATPM.lower_bound = 3.15

sol = mcs.fba(network,constr=["O2Up=0"])
# specify modules
modules = mcs.SD_Module(network,"mcs_lin",module_sense="target")

network.remove_metabolites
network.metabolites.list_attr("compartment")
# specify MCS setup
maxSolutions = 4
maxCost = 50

# construct MCS MILP
mcsEnum = mcs.StrainDesigner(network,modules,max_cost=7,M=None)

# solve MILP
mcsEnum.compute_smallest_mcs(max_solutions=maxSolutions)

Constructing MCS MILP.
Bounding MILP.
Enumerating smallest MCS ...
{'R_ATPM': -1}
{'R_CO2Ex': -1, 'R_h_pEx': -1}
{'R_CYTBO3_4pp': -1, 'R_ENO': -1}
{'R_CYTBO3_4pp': -1, 'R_GAPD': -1}


(<4x82 sparse matrix of type '<class 'numpy.float64'>'
 	with 7 stored elements in Compressed Sparse Row format>,
 0)

In [16]:
import traceback
import warnings
import sys

def warn_with_traceback(message, category, filename, lineno, file=None, line=None):

    log = file if hasattr(file,'write') else sys.stderr
    traceback.print_stack(file=log)
    log.write(warnings.formatwarning(message, category, filename, lineno, line))

warnings.showwarning = warn_with_traceback
warnings.simplefilter("always")

In [11]:
#import ray
#ray.init()

@ray.remote
class timer(object):
    import time
    def __init__(self):
        self.starttime = self.time.monotonic_ns()
    def timepassed(self,x):
        self.time.sleep(0.1)
        return self.time.monotonic_ns()-self.starttime

parpool = ray.util.ActorPool([timer.remote() for _ in range(int(ray.available_resources()['CPU']))])

times = list(parpool.map(lambda a,x: a.timepassed.remote(x),range(20)))

{'__name__': 'mcs', '__doc__': None, '__package__': 'mcs', '__loader__': <_frozen_importlib_external.SourceFileLoader object at 0x0000026A0BAB7EC8>, '__spec__': ModuleSpec(name='mcs', loader=<_frozen_importlib_external.SourceFileLoader object at 0x0000026A0BAB7EC8>, origin='c:\\users\\phili\\documents\\python\\mcs\\mcs\\__init__.py', submodule_search_locations=['c:\\users\\phili\\documents\\python\\mcs\\mcs']), '__path__': ['c:\\users\\phili\\documents\\python\\mcs\\mcs'], '__file__': 'c:\\users\\phili\\documents\\python\\mcs\\mcs\\__init__.py', '__cached__': 'c:\\users\\phili\\documents\\python\\mcs\\mcs\\__pycache__\\__init__.cpython-37.pyc', '__builtins__': {'__name__': 'builtins', '__doc__': "Built-in functions, exceptions, and other objects.\n\nNoteworthy: None is the `nil' object; Ellipsis represents `...' in slices.", '__package__': '', '__loader__': <class '_frozen_importlib.BuiltinImporter'>, '__spec__': ModuleSpec(name='builtins', loader=<class '_frozen_importlib.BuiltinImpor

In [8]:
import time
starttime = time.monotonic_ns()
time.sleep(0.5)
print(time.monotonic_ns()-starttime)

516000000
